#### 중분류명이 제휴사별로 상이하고, 그 종류가 다양하여 분류명으로써 제 기능을 하지 못한다고 판단했습니다. 
#### 따라서 A제휴사의 카테고리 이름을 참고하여 8가지로 그 분류를 재범주화했습니다. 
#### 이를 간략분류(또는 NEW_CAT) 이라는 이름의 칼럼으로 사용했습니다.

In [2]:
import cx_Oracle
import pandas as pd
import numpy as np

In [5]:
db = cx_Oracle.connect("LMEMBER", "LMEMBER", 'localhost:1521/xepdb1')
cursor = db.cursor()
cursor.execute('select * from PRODCL')
x = cursor.fetchall()
category = pd.DataFrame(x)
category

,0,1,2,3,4,5
0,A,4,0409,A040902,디자이너,디자이너부틱
1,A,4,0409,A040903,디자이너,디자이너캐릭터
2,A,4,0410,A041001,모피/피혁,모피
3,A,4,0410,A041002,모피/피혁,피혁
4,A,4,0410,A041003,모피/피혁,숙녀고정행사
...,...,...,...,...,...,...
4381,D,4,0402,D040222,음료,코코아
4382,D,4,0402,D040223,음료,아이스티
4383,D,5,0501,D050101,화장지/티슈,롤티슈
4384,D,5,0501,D050102,화장지/티슈,각티슈/미용티슈


In [8]:
cursor.execute("""select column_name from user_tab_columns where table_name='PRODCL'""")
col = cursor.fetchall()
list1 = []
for i in col:
    for j in i:
        list1.append(j)
category.columns = list1
category.head()

,제휴사,대분류코드,중분류코드,소분류코드,중분류명,소분류명
0,A,4,0409,A040902,디자이너,디자이너부틱
1,A,4,0409,A040903,디자이너,디자이너캐릭터
2,A,4,0410,A041001,모피/피혁,모피
3,A,4,0410,A041002,모피/피혁,피혁
4,A,4,0410,A041003,모피/피혁,숙녀고정행사


B 제휴사 대분류가 너무 세분화되어 있는 것 같다.
카테고리화가 필요할 것 같다.

In [11]:
cursor.close()
db.close()

InterfaceError: not open

In [12]:
mask = category['제휴사']=='A'
cat_A = category[mask]
mask1 = cat_A['중분류명']=='기타'
cat_A[mask1]

,제휴사,대분류코드,중분류코드,소분류코드,중분류명,소분류명
573,A,1,0111,A011101,기타,PECK Restaurant
574,A,1,0111,A011102,기타,PECK Enoteca
575,A,1,0111,A011103,기타,PECK Fruits
576,A,1,0111,A011104,기타,PECK G&C
577,A,1,0111,A011105,기타,팩토리샵
578,A,1,0111,A011106,기타,CJ팩토리샵
579,A,1,0111,A011107,기타,스테이셔너리
580,A,1,0111,A011108,기타,5 ON THE GO
581,A,1,0111,A011109,기타,멘즈셀렉샵


In [13]:
category[category['소분류명']=='통조림선물세트']

,제휴사,대분류코드,중분류코드,소분류코드,중분류명,소분류명
1204,B,12,1207,B120701,인스턴트선물세트,통조림선물세트


In [14]:
category[category['중분류명']=='맛집행사']

,제휴사,대분류코드,중분류코드,소분류코드,중분류명,소분류명
2219,B,63,6306,B630601,맛집행사,테마먹거리
2220,B,63,6306,B630602,맛집행사,브랜드먹거리


In [15]:
import cx_Oracle
import pandas as pd
db = cx_Oracle.connect("LMEMBER","LMEMBER","localhost:1521/xepdb1")
cursor = db.cursor()
cursor.execute("select * from FOOD_ABC")
x = cursor.fetchall()
food_abc = pd.DataFrame(x)
cursor.execute("select column_name from user_tab_columns where table_name='FOOD_ABC'")
col = cursor.fetchall()
list1 = []
for i in col:
    for j in i:
        list1.append(j)
food_abc.columns = list1
food_abc.head()
cursor.close()
db.close()




def categorize(value):
    if value in ['냉장축산','육류', '축산가공', '육포', '마른안주선물세트', '우육', '수입육', '일반한우', '브랜드한우', '닭고기', '계란', '오리고기', '양념육', '돼지고기', '브랜드돼지고기', '차별화돼지고기', '온라인돼지고기', '미국산소고기', '호주산소고기', '수입돼지고기', '기타국소고기','가공란', '계란류', '국내산우육', '국내산돈육', '국내산계육', '수입우육', '수입돈육', '기타축산']:
        return '육류'
    
    elif value in ['냉장농산','건과','LOCAL잎채소','LOCAL김치채소','가공양곡','농산물', '열매채소', '버섯', '온라인버섯/열매채소', '양곡', '잡곡류', '즉석도정미', '잎채소', '뿌리채소', '열매채소', '버섯류', '샐러드채소', '약초', '양념류', '친환경채소', '김치채소', '온라인잎/샐러드채소', 'LOCAL잎채소', '샐러드/간편채소', '가루', '식자재', '나물', '콩나물', '두부', '간편채소', '과실과일', '과채과일', '친환경과일', '감귤', '감', '복숭아', '자두', '토마토', '포도', '참외', '수박', '딸기', '메론', '수입감귤과일', '수입기타과일', '열대과일','키위/석류', '포도/체리', '오렌지', '자몽', '오렌지과일기타', '석류', '레몬', '견과', '블루베리', '체리', '수입포도', '냉동과일', '열대과일기타','바나나', '파인애플', '망고', '키위', '잡곡', '선식', '온라인잡곡선식', '양념채소', '뿌리채소', '온라인양념/뿌리채소','사과', '배', '온라인저장과일', '구색과일', '밤', '국산과일', '수입과일', '냉동과일', '건과일', '견과류', '양곡', '잡곡', '선식류', '엽채류', '과채류', '근채류', '두채류', '김치채소류', '버섯류','양채류', '나물류']:
        return '농산물'
    
    elif value in ['냉장수산','수입갈치','국산게/갈치','수산품', '대중선어', '냉동수산물', '고등어/삼치', '대구', '도미/민어', '기타구색생선', '꽁치/가자미', '오징어', '조기/명태', '국산게', '즉석구이안주', '마른안주', '멸치건명태', '해조류', '특산물', '건오징어', '조미오징어', '쥐치포', '해초류', '생선회', '해물연체류', '간편조리생선','기타회', '즉석구이안주', '특산물행사', '수입연어', '수입고등어', '수입게', '수입기타생선','냉동규격생선', '냉동규격해물', '냉장규격생선', '온라인규격/간편조리생선', '규격RTC', '간편조리행사', '굴비', '반건생선', '해초', '조개', '조개살', '연체', '새우', '온라인해물연체', '특산물행사','멸치', '천연조미료', '황태', '미역/다시마', '즉석구이김', '마른김', '조미김', '김가공품', '선어류', '선어가공', '건어류', '건어가공', '해물류', '기타수산']:
        return '수산물'
    
    elif value in ['튀김조리', '피자', '브랜드빵', '간편조리식품', '간편요리', '밥죽류', '조미료', '소스류', '장류', '식용유지', '조미선물세트', '향신료', '액상조미료', '복합조미료', '소금/설탕류', '수입조미', '즉석참기름', '육가공품', '반찬','식사류','즉석식품','냉장식사MS','젓갈/반찬', '조리반찬', '시즌반찬', '포장젓갈', '포장반찬','반찬류', '구이', '튀김', '밥류', '구이찜조리', '대용식', '피자', '이벤트조리', '간편조리',  '장류', '식용유', '조미료', '케첩마요네즈', '감미료','묵', '묵/즉석참기름', '어묵맛살', '냉장면류', '햄소시지','냉장반찬', '냉동만두', '냉동간식', '냉동대용식','냉장기타MS', '냉동MS',  '햄', '소시지', '냉장햄세트','생선초밥', '냉동가공', '밥류', '김/초밥','도시락', '간편조리', '떡', '족발', '순대', '단무지류', '김치류', '반찬류', '젓갈류','먹거리', '브랜드푸드몰', '포장반찬HMR', '냉동HMR', '냉장간편식HMR', '냉장식사HMR']:
        return '식사류/반찬'
    
    elif value in ['건강식품','생활한방', '한방식재류', '한방차', '비타민', '기능성건강', '홍인삼']:
        return '건강식품'
    
    elif value in ['유제품','간식','가공식품', '베이커리', '치즈', '디저트', '간식조리', '유가공', '라면', '면류', '분유', '잼', '통조림', '이유식/유아간식', '꿀', '스낵', '비스켓', '파이', '시리얼', '캔디', '초콜릿', '껌', '안주', '한과', '프리미엄과자', '간식용과자', '수입과자', '종합과자', '전통과자', '아이스크림','간식조리','빵', '전통간식', '시즌행사', '맛집행사', '라면', '면류', '브랜드빵', '시리얼', '간편간식', '베이커리', '기타조리식품', '치즈버터', '아이스크림', '만두', '냉장간식MS', '병통조림', '분유', '가공건식품', '가공분말류', '라면', '건면', '쿠키케이크','스낵', '캔디껌류', '초콜렛', '전통과자', '씨리얼']:
        return '간식류'
    
    elif value in ['담배','주류', '소주', '맥주', '와인', '전통주', '양주', '소주', '맥주', '와인', '양주', '전통주', '기타음주류']:
        return '주류'
    
    elif value in ['차/커피','냉장음료', '흰우유', '가공우유', '요구르트', '두유', '커피', '차', '인스턴트커피', '원두커피', '시즌차', '생수', '과채음료', '두유', '탄산음료', '이온음료', '커피차음료', '기능성음료', '수입음료','우유', '발효유', '냉장음료', '생수', '탄산음료', '과채음료', '커피음료', '차음료', '기능성음료', '두유', '커피류', '차류']:
        return '음료'
    
    else:
        return ' '





food_abc['간략분류'] = food_abc['중분류명'].apply(lambda x:categorize(x))





def gift_set_cat(cat_value ,s_value):
    if s_value in ['선어세트','해물선물세트','멸치선물세트' ,'마른안주 선물세트', '해조류세트', '멸치황태세트', '규격갈치선물세트', '규격옥돔선물세트', '규격고등어선물세트', '규격기타생선선물세트', '굴비세트', '기타해물선물세트', '전복선물세트', '멸치세트', '김선물세트']:
        return '수산물'
    elif s_value in ['오렌지과일선물세트','열대과일선물세트','배세트', '사과배세트', '사과세트','친환경과일세트', '혼합세트', '배세트', '혼합세트', '사과세트', '기타과일세트', 'PB세트', '친환경과일선물세트', '키위세트', '기타수입과일세트', '견과선물세트', '약용버섯세트', '더덕세트']:
        return '농산물'
    elif s_value in ['피자', '냉장간편식','냉장장류','온라인냉장반찬HMR','반찬류행사','양념류선물세트']:
        return '식사류/반찬'
    elif s_value in ['인삼세트', '한방세트', '더덕세트', '건강기능성선물세트', '홍삼선물세트']:
        return '건강식품'
    elif s_value in ['디저트','통조림선물세트', '혼합선물세트','온라인 선물세트','유제품', '화과자', '과자선물세트', '일반간식류행사']:
        return '간식류'
    elif s_value in ['한우냉동선물세트','한우냉장선물세트','수입냉동선물세트', '수입냉장선물세트','한우선물세트','수입소고기선물세트', '햄선물세트']:
        return '육류'
    elif s_value in ['커피선물세트','차선물세트']:
        return '음료'
    else:
        return cat_value





food_abc['간략분류'] = food_abc.apply(lambda x:gift_set_cat(x['간략분류'], x['소분류명']), axis=1)





# mask 씌울 항목 선정
mask = food_abc['중분류명'].isin(['기타', '브랜드조리','데일리처분','분식','양식','일식','중식','한식','패스트푸드','레스토랑','카페','푸드코트','컨세션']) 
food_abc = food_abc[~mask] # ~를 포함하게 되면 mask의 값을 제외, ~을 제외하면 mask의 값을 포함입니다.






# food_abc.to_csv('dataset/food_abc.csv', sep=',', na_rep='NaN',encoding='utf-8-sig')

In [16]:
food_abc.head()

,제휴사,대분류코드,중분류코드,소분류코드,중분류명,소분류명,간략분류
0,B,14,1413,B141301,수입과자,수입껌,간식류
1,B,14,1413,B141302,수입과자,수입비스켓,간식류
2,B,14,1413,B141303,수입과자,수입스낵,간식류
3,B,14,1413,B141304,수입과자,수입초콜릿,간식류
4,B,14,1413,B141305,수입과자,수입캔디,간식류


In [ ]:
food_abc['간략분류'].unique()

In [ ]:
food_abc.isnull().sum()